In [1]:
!pip install streamlit pyngrok PyPDF2 transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.5 MB/s eta 0:00:00


In [2]:
code = '''
import streamlit as st
from PyPDF2 import PdfReader
from transformers import pipeline

qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

def read_pdf(file):
    reader = PdfReader(file)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\\n"
    return text

st.title("PDF Chat Application")
st.write("Upload a PDF document to interact with it.")

uploaded_file = st.file_uploader("Choose a PDF file", type="pdf")

if uploaded_file:
    pdf_text = read_pdf(uploaded_file)
    st.write("PDF content loaded.")

    if "conversation" not in st.session_state:
        st.session_state.conversation = []

    for question, answer in st.session_state.conversation:
        st.write(f"**Q:** {question}")
        st.write(f"**A:** {answer}")

    user_question = st.text_input("Ask a question about the PDF:")

    if st.button("Submit Question"):
        if user_question:
            answer = qa_pipeline(question=user_question, context=pdf_text)
            st.session_state.conversation.append((user_question, answer['answer']))

            if answer['score'] > 0.5:
                context_snippet = pdf_text[max(0, answer['start'] - 50):min(len(pdf_text), answer['end'] + 50)]
                full_answer = f"The answer is: {answer['answer']}. Here is some context: {context_snippet.strip()}"
            else:
                full_answer = "I'm sorry, I couldn't find a relevant answer in the PDF."

            st.write(f"**Q:** {user_question}")
            st.write(full_answer)
'''

with open("app.py", "w") as f:
    f.write(code)


In [4]:
!ngrok config add-authtoken 2vlnWXWMdgCCYk89NEtCE4INpfm_3QTyPVwMZ2mc2mwAGctK3


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [6]:
from pyngrok import ngrok
!streamlit run app.py &>/dev/null &

# Get the public URL
# Change 'port' to 'addr' and pass the port number as a string
public_url = ngrok.connect(addr="8501")
print(f"Streamlit app is live at: {public_url}")

Streamlit app is live at: NgrokTunnel: "https://7b8b-34-106-87-203.ngrok-free.app" -> "http://localhost:8501"
